# Humidity Controll

In [29]:
%reset -f

In [30]:
import numpy as np
import matplotlib.pyplot as plt
import psychro as psy
import pandas as pd
import va_hum

![AHU](cool_CAV.svg)

In [31]:
# constants
l = 2496e3                  # specific latent heat of vaporization [J/kg]
c_da, c_v  = 1e3, 1.96e3    # J/(kg·K), specific heats of dry air and water vapor
ρ = 1.2                     # kg/m3, density of air

# parameters
ϕ_o = 0.8          # relative humidity of outside air [-]
θ_o = 20            # temperature of outside Air [°C]
θ_6_sp = 22         # temperature setpoint of the room [°C]
ϕ_6_sp = 0.6        # relative humidity setpoint of the room [-]

UA = 936 # total transmittance [W/K]
m_i = 2.120 # infiltration mass flow [kg/s]
Q_aux_s = 4000 # auxiliary heat sensible [W]
Q_aux_l = 3000 # auxiliary  latent gain [W]


In [32]:
# humidity ratios
w_6_sp = psy.w(θ_6_sp, ϕ_6_sp)  # humidity ratio setpoint of the room [kg/kg]
w_out = psy.w(θ_o, ϕ_o)  # humidity ratio of outside air [kg/kg]

In [ ]:
 # Define column names (unknowns)
columns = [
    'θ1', 'w1', 'θ2', 'w2', 'θ3', 'w3', 'θ4', 'w4','θ5', 'w5','θ6', 'w6',
    'Qs_CC', 'Ql_CC', 'Qt_CC', 'Qs_HC', 'Ql_VH', 'Qs_TZ', 'Ql_TZ'
]

# Define row names (equations) with 's' for sensible and 'l' for latent
rows = [
    'MXs', 'MXl',                    # Mixing box equations
    'CCs', 'CCl', 'DH', 'DH_Q',      # Colling with dehumidification
    'MX2s', 'MX2l',                  # Mixing after dehumidification
    'HCs', 'HCl',                    # Heating coil equations
    'VHs', 'VHl',                    # Vapor humidifier equations

    'TZs', 'TZl',                   # Thermal zone equations
    'BLs', 'BLl',                   # Building equations
    'Kθ', 'Kw'                      # Controller equations
]

# Initialize coefficient matrix A and vector b
A_DH = pd.DataFrame(0.0, index=rows, columns=columns)
b_dh = pd.Series(0.0, index=rows)


In [36]:
A

,θ1,w1,θ2,w2,θ3,w3,θ4,w4,θ5,w5,θ6,w6,Qs_CC,Ql_CC,Qt_CC,Qs_HC,Ql_VH,Qs_TZ,Ql_TZ
MXs,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
MXl,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
CCs,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
CCl,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
DH,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
DH_Q,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
MX2s,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
MX2l,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
HCs,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
HCl,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
